<a href="https://colab.research.google.com/github/venomouscyanide/dl_sain/blob/master/week1/mlp_week1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import pickle
import gzip

# Third-party libraries
import numpy as np

def load_data():
    """Return the MNIST data as a tuple containing the training data,
    the validation data, and the test data.
    The ``training_data`` is returned as a tuple with two entries.
    The first entry contains the actual training images.  This is a
    numpy ndarray with 50,000 entries.  Each entry is, in turn, a
    numpy ndarray with 784 values, representing the 28 * 28 = 784
    pixels in a single MNIST image.
    The second entry in the ``training_data`` tuple is a numpy ndarray
    containing 50,000 entries.  Those entries are just the digit
    values (0...9) for the corresponding images contained in the first
    entry of the tuple.
    The ``validation_data`` and ``test_data`` are similar, except
    each contains only 10,000 images.
    This is a nice data format, but for use in neural networks it's
    helpful to modify the format of the ``training_data`` a little.
    That's done in the wrapper function ``load_data_wrapper()``, see
    below.
    """
    f = gzip.open('/content/drive/MyDrive/Colab Notebooks/mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = pickle.load(f, encoding="latin1")
    f.close()
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    """Return a tuple containing ``(training_data, validation_data,
    test_data)``. Based on ``load_data``, but the format is more
    convenient for use in our implementation of neural networks.
    In particular, ``training_data`` is a list containing 50,000
    2-tuples ``(x, y)``.  ``x`` is a 784-dimensional numpy.ndarray
    containing the input image.  ``y`` is a 10-dimensional
    numpy.ndarray representing the unit vector corresponding to the
    correct digit for ``x``.
    ``validation_data`` and ``test_data`` are lists containing 10,000
    2-tuples ``(x, y)``.  In each case, ``x`` is a 784-dimensional
    numpy.ndarry containing the input image, and ``y`` is the
    corresponding classification, i.e., the digit values (integers)
    corresponding to ``x``.
    Obviously, this means we're using slightly different formats for
    the training data and the validation / test data.  These formats
    turn out to be the most convenient for use in our neural network
    code."""
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = zip(training_inputs, training_results)
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    """Return a 10-dimensional unit vector with a 1.0 in the jth
    position and zeroes elsewhere.  This is used to convert a digit
    (0...9) into a corresponding desired output from the neural
    network."""
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e


In [16]:
from typing import List, Tuple
import numpy as np

In [17]:
class NetworkUtils:
    @staticmethod
    def sigmoid(z: np.ndarray) -> np.ndarray:
        return 1 / (1 + np.exp(-z))

    @staticmethod
    def sigmoid_prime(z: np.ndarray) -> np.ndarray:
        return NetworkUtils.sigmoid(z) * (1 - NetworkUtils.sigmoid(z))

In [18]:
class Network:
    def __init__(self, training_data: List[Tuple[np.ndarray, np.ndarray]],
                 testing_data: List[Tuple[np.ndarray, int]],
                 size: List[int], learning_rate: float, epochs: int,
                 mini_batch_size: int):
        self.training_data = training_data
        self.testing_data = testing_data
        self.size = size
        self.num_layers = len(size)
        self.learning_rate = learning_rate
        self.biases = []
        self.weights = []
        self._init_biases()
        self._init_weights()
        self.epochs = epochs
        self.mini_batch_size = mini_batch_size

    def _init_biases(self):
        for i in range(1, self.num_layers):
            self.biases.append(np.random.randn(self.size[i], 1))

    def _init_weights(self):
        bias_matrix_sizes = [(self.size[x + 1], self.size[x]) for x in range(self.num_layers - 1)]
        for x, y in bias_matrix_sizes:
            self.weights.append(np.random.randn(x, y))

    def train(self):
        for epoch in range(self.epochs):
            np.random.shuffle(self.training_data)
            print(f"Start training for epoch: {epoch + 1} of {self.epochs}")

            num_mini_batches = len(self.training_data) // self.mini_batch_size
            mini_batches = self._create_mini_batches()

            for batch, mini_batch in enumerate(mini_batches, start=1):
                if batch % 1000 == 0:
                    print(f"calculating SGD for: Batch {batch}/{num_mini_batches} of epoch: {epoch}")
                self._update_b_w(mini_batch)
            self._calc_accuracy()

    def _create_mini_batches(self) -> List[List[Tuple[np.ndarray, np.ndarray]]]:
        mini_batches = [
            self.training_data[multiple:multiple + self.mini_batch_size] for multiple in
            range(0, len(self.training_data), self.mini_batch_size)
        ]
        return mini_batches

    def _update_b_w(self, mini_batch: List[Tuple[np.ndarray, np.ndarray]]):
        nabla_bias = self._get_nabla_bias_zeroes()
        nabla_wt = self._get_nabla_wt_zeroes()

        for x, y in mini_batch:
            del_bias, del_wt = self._run_back_propagation(x, y)

            nabla_bias = [curr_b + del_b for curr_b, del_b in zip(nabla_bias, del_bias)]
            nabla_wt = [curr_wt + del_w for curr_wt, del_w in zip(nabla_wt, del_wt)]

        self.biases = [
            b - ((self.learning_rate / self.mini_batch_size) * nb) for b, nb in zip(self.biases, nabla_bias)
        ]
        self.weights = [
            w - ((self.learning_rate / self.mini_batch_size) * nw) for w, nw in zip(self.weights, nabla_wt)
        ]

    def _get_nabla_bias_zeroes(self) -> List[np.ndarray]:
        return [np.zeros(np.shape(bias)) for bias in self.biases]

    def _get_nabla_wt_zeroes(self) -> List[np.ndarray]:
        return [np.zeros(np.shape(wt)) for wt in self.weights]

    def _run_back_propagation(self, x: np.ndarray, y: np.ndarray) -> Tuple[List[np.ndarray], List[np.ndarray]]:
        nabla_bias = self._get_nabla_bias_zeroes()
        nabla_wt = self._get_nabla_wt_zeroes()

        activations = []
        z_list = []

        a = x
        activations.append(a)

        for i in range(self.num_layers - 1):
            z = np.dot(self.weights[i], a) + self.biases[i]
            z_list.append(z)

            a = NetworkUtils.sigmoid(z)
            activations.append(a)

        error_l = np.multiply(self._nabla_a(activations[-1], y), NetworkUtils.sigmoid_prime(z_list[-1]))
        nabla_bias[-1] = error_l
        nabla_wt[-1] = np.dot(error_l, np.transpose(activations[-2]))

        for layer in range(self.num_layers - 2, 0, -1):
            error_l = np.multiply(
                np.dot(np.transpose(self.weights[layer]), error_l), NetworkUtils.sigmoid_prime(z_list[layer - 1])
            )

            nabla_bias[layer - 1] = error_l
            nabla_wt[layer - 1] = np.dot(error_l, activations[layer - 1].transpose())

        return nabla_bias, nabla_wt

    def _nabla_a(self, a_l: np.ndarray, y: np.ndarray) -> np.ndarray:
        return a_l - y

    def _calc_accuracy(self):
        correct_results = 0
        total_results = len(self.testing_data)
        for x, y in self.testing_data:
            logit = self.feedforward(x)
            if np.argmax(logit) == y:
                correct_results += 1
        print(f"Total accuracy on testing data: {round((correct_results / total_results) * 100, 2)}")

    def feedforward(self, x: np.ndarray) -> np.ndarray:
        a = x
        for layer in range(self.num_layers - 1):
            a = NetworkUtils.sigmoid(np.dot(self.weights[layer], a) + self.biases[layer])
        return a

In [25]:
class Hyperparameters:
    SIZE: List[int] = [28 * 28, 30, 10]
    LEARNING_RATE: float = 10
    EPOCHS: int = 15
    MINI_BATCH_SIZE: int = 10

In [26]:
def train_and_eval():
    training, _, testing = load_data_wrapper()
    params = Hyperparameters
    mlp = Network(list(training), list(testing), params.SIZE, params.LEARNING_RATE, params.EPOCHS,
                  params.MINI_BATCH_SIZE)
    mlp.train()

In [ ]:
train_and_eval()

Start training for epoch: 1 of 15
calculating SGD for: Batch 1000/5000 of epoch: 0
calculating SGD for: Batch 2000/5000 of epoch: 0
calculating SGD for: Batch 3000/5000 of epoch: 0
calculating SGD for: Batch 4000/5000 of epoch: 0
calculating SGD for: Batch 5000/5000 of epoch: 0
Total accuracy on testing data: 84.41
Start training for epoch: 2 of 15
calculating SGD for: Batch 1000/5000 of epoch: 1
calculating SGD for: Batch 2000/5000 of epoch: 1
calculating SGD for: Batch 3000/5000 of epoch: 1
calculating SGD for: Batch 4000/5000 of epoch: 1
calculating SGD for: Batch 5000/5000 of epoch: 1
Total accuracy on testing data: 89.57
Start training for epoch: 3 of 15
calculating SGD for: Batch 1000/5000 of epoch: 2
calculating SGD for: Batch 2000/5000 of epoch: 2
calculating SGD for: Batch 3000/5000 of epoch: 2
calculating SGD for: Batch 4000/5000 of epoch: 2
calculating SGD for: Batch 5000/5000 of epoch: 2
Total accuracy on testing data: 93.02
Start training for epoch: 4 of 15
calculating SGD